In [ ]:
## Datos de la entrega

Número de grupo

Integrantes : Nombre y cédula

# Procesamiento de textos, parte 1

*(Cada uno de los párrafos numerados requiere de la inserción a continuación de __al menos una__ celda, ya sea de tipo código o texto) *

Realizaremos procesamiento de textos sobre textos en inglés. El objetivo es determinar el tema del documento, se habla de **categorizar** el documento. Utilizaremos un conjunto de documentos, incluido en la distribución de sklearn, conocido con el nombre **20 Newsgroups** (qwone.com/~jason/20Newsgroups/ ). Se trata de un conjunto de aproximadamente 18.000 mensajes de correo, distribuidos en 20 temas distintos, tales como **religión cristiana**, **ateísmo**, **armas**, **mac hardware**, **computación gráfica**, etc.  Estos temas, a su vez, pueden agruparse entre sí con distintos grados de cercanía. El conjunto ha sido muy utilizado en todo tipo de experimentos de aprendizaje automático y se encuentran disponibles varias implementaciones.

En esta tarea se pide que
* Lea la documentación presente en la guía del usuario de scikit-learn, sección **5.7** y la documentación en el sitio web, previamente mencionado, sobre el conjunto de datos
* Conteste algunas preguntas vinculadas
* Realice categorización de subconjuntos del conjunto de datos


##1  

Inicialice su ambiente importando los módulos necesarios
Cargue los datos de entrenamiento del dataset **20 Newsgroups**, con un módulo ya disponible en sklearn, 
seleccionando 5 categorías distintas, tales que 3 de ellas se encuentren separadas (temas no relacionados) según el esquema del sitio web de 20 Newsgroups y 3 de ellas en el mismo bloque(temas próximos).
Despliegue las categorías de los documentos importados

In [2]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
from datetime import datetime as dt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [3]:
cats = ['rec.motorcycles', 'misc.forsale', 'sci.space','sci.crypt','sci.electronics']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)
pprint(list(newsgroups_train.target_names))

['misc.forsale',
 'rec.motorcycles',
 'sci.crypt',
 'sci.electronics',
 'sci.space']


##2

¿Cuántos archivos tiene su dataset?
Imprima, del 4to archivo, el tema y las 30 primeras líneas.
Imprima las categorías de los 1eros 20 documentos

In [8]:
#Cantidad de archivos en el conjunto de datos
newsgroups_train.target.shape[0]

2962

In [9]:
#Tema del 4to archivo
print newsgroups_train.target_names[newsgroups_train.target[3]]

sci.space


In [10]:
#Primeras 30 líneas
print("\n".join(newsgroups_train.data[3].split("\n")[:30]))

From: prb@access.digex.com (Pat)
Subject: Re: Jemison on Star Trek
Organization: Express Access Online Communications USA
Lines: 14
NNTP-Posting-Host: access.digex.net

In article <1993Apr22.214735.22733@Princeton.EDU> phoenix.Princeton.EDU!carlosn (Carlos G. Niederstrasser) writes:
>A transporter operator!?!?  That better be one important transport.  Usually  
>it is a nameless ensign who does the job.  For such a guest appearance I would  
>have expected a more visible/meaningful role.


Christian  Slater, only gota  cameo on ST6,  

and besides.

Maybe she can't act:-)

pat




In [11]:
#Categorías de los primeros 20 documentos
for t in newsgroups_train.target[:20]:
    print(newsgroups_train.target_names[t])

sci.crypt
sci.electronics
sci.crypt
sci.space
misc.forsale
sci.space
sci.space
sci.space
sci.electronics
misc.forsale
sci.crypt
sci.space
sci.electronics
sci.crypt
sci.space
sci.crypt
sci.electronics
rec.motorcycles
misc.forsale
misc.forsale


##3 

En cada documento, que es un correo electrónico, hay metadatos (quien envió el correo, fecha, organización, etc.). ¿Los metadatos pueden incidir en la categorización de un documento?
¿Por qué puede ser conveniente sacarlos?

Los metadatos pueden influir en la categorización. Dado que los usuarios en general se especializan en pocas temáticas a la hora de realizar documentos, su email podria ser un atributo importante para descartar algunas categorías, también pueden influir negativamente: una dirección de email puede escribir habitualmente sobre un tema y esporádicamente sobre otro, sin embargo ésto no debería pesar en la categorización. Lo anterior es un caso de overfitting de los datos. Las etiquetas de los metadatos que aparecen siempre como "Subject" y "From" pueden influir en menor medida.

Creemos que como los metadatos no forman parte del documento en sí y pueden ser contraproducentes a la hora de evaluar su categoría, conviene evaluar una instancia estrictamente por su información y no por la información secundaria que puede o no estar relacionada, por lo tanto es conveniente eliminar los metadatos.

##4 

Genere una nueva versión del conjunto de datos, similar al ya generado, pero sin metadatos. Utilice para ello funcionalidades definidas en sklearn.

In [12]:
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats, remove=('headers', 'footers', 'quotes'))
#4to documento sin metadatos
print newsgroups_train.data[3]



Christian  Slater, only gota  cameo on ST6,  

and besides.

Maybe she can't act:-)


##5 

Un modo de utilizar los datos de texto en aprendizaje es pasar a un modelo vectorial. Explique brevemente como es el modelo vectorial con *bag-of-words*, qué hace *CountVectorizer* definido en sklearn, qué es *tf-idf* y cómo se implementa en Python. 

La extracción de atributos consiste en transformar datos en general como texto o imágenes en atributos numéricos utilizables en métodos de aprendizaje automático. Cuando se trata de texto resulta imposible introducir una secuencia de simbolos de largo variable en los algoritmos de aprendizaje automático ya que la mayoría de ellos espera vectores de atributos númericos de tamaño fijo. La alternativa consiste en tokenizar los textos de forma de dividirlos en simbolos independientes llamados tokens utilizando los espacios y la puntuación, para posteriormente contar las ocurrencias de dichos tokens y normalizar el resultado. Cada token será considerado un atributo y la representación se denomina **Bag-Of-Words**. Dicha representación no considera el orden original de los tokens en el texto. **Count Vectorizer** es una implementación de extración de atributos de texto que realiza la tokenización y el conteo de las ocurrencias de cada token, **TfidfVectorizer** realiza además la normalización de dicho conteo: sucede que existen cierto tipo de palabras como los artículos que se repiten en la gran mayoría de los textos y que tienen poco significado real en el significado del texto, sin normalización el conteo de éste tipo de palabras podrían restar importancia de otras palabras menos frecuentes pero más significativas en los algoritmos utilizados.
El **tf-idf** es una medida para la importancia de una palabra en un documento perteneciente a un corpus mas grande de documentos. Se calcula como el producto de ortras dos medidas, el **tf**, que es la frecuencia con la que la palabra aparece en el documento, y el **idf**, que se calcula en base a la inversa de la cantidad de documentos del corpus en las que aparece la palabra. De esta forma se premian las palabras que aparecen varias veces, pero en pocos documentos.

Para implementar **TfidfVectorizer** se utiliza una función *fit_transform(..)* la cuál ajusta el estimador del método a los datos y transforma la matriz mencionada anteriormente en otra representación más adecuada al método.

##6 

Repita el proceso de creación que aparece en el tutorial, llegando a un array 
que contiene el modelo vectorial para los datos actuales (los documentos seleccionados del dataset original)
¿Cuál es el tamaño del vocabulario? ¿Cuál es el largo máximo de un documento? 
¿Qué conclusiones puede sacar sobre el array que representa a un documento?

In [13]:
count_vect = CountVectorizer()    
vec_vocab = count_vect.fit_transform(newsgroups_train.data)
#Tamaño del vocabulario
print str(vec_vocab.shape[1])

31901


In [20]:
#convierto vocabulario en arreglo
voc_arr = vec_vocab.toarray()
#sumo cada fila para obtener la cantidad de palabras por documento
dist = np.sum(voc_arr, axis=1)
#imprimo sumatoria por cada documento
print dist
#Largo del documento con mayor cantidad de palabras
print max(dist)

[ 37   0 128 ...,  30   3  12]
11440


##7 

Resuelva el problema de la categorización con algún algoritmo de aprendizaje (tratando de obtener buenos resultados). Calcule e imprima la medida-F, imprima la matriz de confusión.

In [24]:
newsgroups_test = fetch_20newsgroups(subset='test',categories=cats,remove=('headers'))

In [25]:
#Vectorizacion
count_vect = CountVectorizer()
count_vect2 = CountVectorizer(min_df=1)
tidf_vect = TfidfVectorizer()
hash_vect = HashingVectorizer(non_negative=True)
Vectorizadores = [    
    ("TfidfVectorizer", tidf_vect),
    ("CountVectorizer", count_vect),
    ("HashingVectorizer", hash_vect)
]

In [26]:
#Comparación de diferentes clasificadores
#Fuente: http://scikit-learn.org/stable/auto_examples/calibration/plot_compare_calibration.html

# Creación de clasificadores
multiNB = MultinomialNB(alpha=.01)
sgd = SGDClassifier()
perceptron = Perceptron()
passiveaggclass = PassiveAggressiveClassifier()
passiveaggclass2 = PassiveAggressiveClassifier(loss='hinge',C=1.0)
passiveaggclass3 = PassiveAggressiveClassifier(loss='squared_hinge',C=1.0)
forest = RandomForestClassifier(n_estimators = 100) 
knn = KNeighborsClassifier()

Clasificadores = [    
    ("KNeighbors", knn),
   # ("RandomForest", forest),
    ("MultinomialNB", multiNB),
    #("SGD", sgd),
    #("Perceptron", perceptron)
]

In [39]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
resultados = []

#Funcion que recibe 
#X conj entrenamiento
#Y conj test
#V conj vectorizadores
#C conj clasificadores
def run(train_data,train_target,test_data,test_target,V,C,matrix = True):
    for nomvect, vectorizador in V:
        #Vectorizo
        X_train = vectorizador.fit_transform(train_data)        
        X_test = vectorizador.transform(test_data)
        for nomclf,clasificador in C:
            print "Metrica F1 con los siguientes parametros"
            print "Clasificador: " + nomclf + ", Vectorizador: " + nomvect   
            if nomclf == 'RandomForest':
                X_train = X_train.toarray()
                X_test = X_test.toarray()
            #Entreno
            clasificador.fit(X_train, train_target)
            #Predigo Test
            prediccion = 0
            prediccion = clasificador.predict(X_test)  
            #Resultados
            print metrics.f1_score(newsgroups_test.target, prediccion, average='weighted')
            if (matrix):
                print confusion_matrix(newsgroups_test.target,prediccion)

In [40]:
#Evaluo resultados
run(newsgroups_train.data,newsgroups_train.target,newsgroups_test.data,newsgroups_test.target,Vectorizadores,Clasificadores)

Metrica F1 con los siguientes parametros
Clasificador: KNeighbors, Vectorizador: TfidfVectorizer
0.235169465248
[[ 70 105  66  86  63]
 [ 47 125  64  95  67]
 [ 51 112  98  82  53]
 [ 51 100  70 107  65]
 [ 55 114  80  76  69]]
Metrica F1 con los siguientes parametros
Clasificador: MultinomialNB, Vectorizador: TfidfVectorizer
0.90550855421
[[346  20   2  12  10]
 [  4 376   2   9   7]
 [  4   1 372  10   9]
 [ 12  13  40 319   9]
 [  1   5   9   6 373]]
Metrica F1 con los siguientes parametros
Clasificador: KNeighbors, Vectorizador: CountVectorizer
0.410092128624
[[196  80  54  32  28]
 [ 19 268  44  30  37]
 [ 20  97 212  22  45]
 [ 45 113 116  69  50]
 [ 19 123 134  21  97]]
Metrica F1 con los siguientes parametros
Clasificador: MultinomialNB, Vectorizador: CountVectorizer
0.907873386434
[[346  19   2  10  13]
 [  4 386   2   2   4]
 [  2   5 375   6   8]
 [ 14  11  41 316  11]
 [  2   7  12   5 368]]
Metrica F1 con los siguientes parametros
Clasificador: KNeighbors, Vectorizador: Ha

##8 

Colapse los documentos con temas relacionados entre sí, asignádoles una única categoría con un nombre adecuado. Resuelva nuevamente el problema de la categorización, con el mismo algoritmo que en el paso anterior y los mismos cálculos de eficiencia. Compare y discuta los resultados. 

In [28]:
news_train = fetch_20newsgroups(subset='train',remove=('headers', 'footers', 'quotes'), categories=cats)
news_test = fetch_20newsgroups(subset='test',remove=('headers', 'footers', 'quotes'), categories=cats)
newsgroup = [news_train, news_test]

In [33]:
for news in newsgroup:
    for t in news.target:
        #print(news.target_names[t])
        if(news.target_names[t] in ["sci.crypt","sci.electronics","sci.space"]):
            news.target_names[t]="science"
            #print(news.target_names[t])

In [41]:
#Evaluo resultados
run(news_train.data,news_train.target,news_test.data,news_test.target,Vectorizadores,Clasificadores)

Metrica F1 con los siguientes parametros
Clasificador: KNeighbors, Vectorizador: TfidfVectorizer
0.236478107524
[[ 70  98  87  80  55]
 [ 42 144  74  83  55]
 [ 58 104  86  84  64]
 [ 59 105  72  95  62]
 [ 36  98  80 102  78]]
Metrica F1 con los siguientes parametros
Clasificador: MultinomialNB, Vectorizador: TfidfVectorizer
0.838706946736
[[335  23   3  18  11]
 [  8 358   9  13  10]
 [  4  23 347  13   9]
 [ 21  30  52 274  16]
 [  0  23  20  10 341]]
Metrica F1 con los siguientes parametros
Clasificador: KNeighbors, Vectorizador: CountVectorizer
0.342850407415
[[171  98  57  38  26]
 [ 53 196  68  41  40]
 [ 34 105 178  32  47]
 [ 52 108 116  54  63]
 [ 26 115 120  34  99]]
Metrica F1 con los siguientes parametros
Clasificador: MultinomialNB, Vectorizador: CountVectorizer
0.832685624481
[[331  24   2  21  12]
 [  9 364   3  16   6]
 [  2  29 342  12  11]
 [ 15  31  50 278  19]
 [  0  34  20  13 327]]
Metrica F1 con los siguientes parametros
Clasificador: KNeighbors, Vectorizador: H

# Procesamiento de textos, parte 2

En esta parte se utilizará otro conjunto muy conocido de datos para aprendizaje, que consiste también de textos en inglés. Se trata de un conjunto de noticias de prensa de la Agencia Reuters, *Reuters-21578 *, https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html

Las noticias han sido categorizadas a mano, con etiquetas de varios tipos, incluyendo *temas, lugares, organizaciones, personas* y otros criterios. Están en formato SGML, las etiquetas aparecen embebidas en el texto. Existe documentación en el archivo README de la distribución.

Nos interesaremos en el tipo de etiqueta *temas (en inglés en los textos topics)*. El objetivo es nuevamente aprender los temas a partir de los textos de las noticias, con la diferencia de que este es un problema multi-etiqueta : cada noticia puede tener varios temas. En vez de resolver el problema multi-etiqueta inicial (con más de 100 tópicos distintos) les pedimos que lo transformen según las siguientes simplificaciones: considere solo los 3 temas más frecuentes, y transforme el problema multi-etiqueta en 3 problemas de clasificación binaria

No se realiza para esta parte una especificación detallada, sino que les pedimos a Uds. que armen los pasos de una solución y definan el detalle del notebook para esta parte. Mínimamente se espera que procesen la entrada SGML, y encuentren algún modo de enfocar el problema multi-etiqueta en la versión simplificada. Se debe proponer algún modo de tratar el texto, con eventuales mejoras respecto a la vectorización de la parte 1. Se deben aplicar clasificadores y medir su performance, mínimamente con precision, recall y medida-F. Finalmente, se espera una discusión detallada de todo lo realizado y eventuales propuestas de mejora.